In [ ]:
import os
import geopandas as gpd
import rasterio
from shapely.geometry import mapping

# Load shapefile
shapefile_path = '/home/shuruyi/Documents/qgis/google_satelite/shapefile/pinus.shp'
shapes = gpd.read_file(shapefile_path)

# Load the image
image_path = '/home/shuruyi/Documents/qgis/google_satelite/google_satelite.tif'
image = rasterio.open(image_path)

# Output directory to save the extracted images
output_dir = '/home/shuruyi/Documents/qgis/google_satelite/extracted_images'

# Ensure the output directory exists or create it if not
os.makedirs(output_dir, exist_ok=True)

# Iterate through each shape and extract ROIs
for index, shape in shapes.iterrows():
    geometry = shape['geometry']
    
    # Check if the geometry is valid
    if geometry is not None:
        shape_geom = [mapping(geometry)]

        # Use rasterio to crop the image based on the shape's geometry
        out_image, out_transform = rasterio.mask.mask(image, shape_geom, crop=True)

        # Generate a unique filename for each extracted ROI
        output_filename = f"extracted_roi_{index}.tif"

        # Create the full output path
        output_path = os.path.join(output_dir, output_filename)

        # Write the extracted ROI to a new file
        with rasterio.open(output_path, 'w', **image.meta) as dest:
            dest.write(out_image)
    else:
        print(f"Skipping shape at index {index} due to invalid geometry.")